In [1]:
from getpass import getpass
OPENAI_API_KEY = getpass("Open API key")

Open API key ········


In [3]:
from PyPDF2 import PdfReader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    openai_api_key = OPENAI_API_KEY,
    temperature = 0,
    model = 'gpt-3.5-turbo'
)

from langchain.schema import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="You are a helpful assistant who "),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

/Users/shiveshkodali/anaconda3/envs/env1/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [5]:
res = chat(messages)
print(res.content)

/Users/shiveshkodali/anaconda3/envs/env1/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


String theory is a theoretical framework in physics that aims to describe the fundamental nature of particles and their interactions. It suggests that the fundamental building blocks of the universe are not point-like particles, but tiny, vibrating strings.

Here are some key points to understand about string theory:

1. Fundamental particles: In string theory, particles are not considered to be point-like objects, but rather tiny, one-dimensional strings. These strings can vibrate in different modes, and the different vibrational patterns give rise to different particles with varying properties, such as mass and charge.

2. Extra dimensions: String theory requires the existence of additional spatial dimensions beyond the three dimensions of space we are familiar with. These extra dimensions are compactified or curled up at extremely small scales, making them undetectable in our everyday experience.

3. Unification of forces: String theory attempts to unify all the fundamental forces o

In [6]:
file_path = 'MSDS_RESUME_TEMPLATE.pdf'

In [7]:
with open(file_path, 'rb') as file:   
    pdf_reader = PdfReader(file)
    
    text = ""
    for page in pdf_reader.pages:
        text+=page.extract_text()
        
text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n",'\n',' ', ''],
            chunk_size = 200,
            chunk_overlap = 20,
            length_function = len
)

In [8]:
chunks = text_splitter.split_text(text)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
knowledge_base = FAISS.from_texts(chunks, embeddings)

/Users/shiveshkodali/anaconda3/envs/env1/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [9]:
question = "Whos resume is this?"

In [10]:
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate

In [11]:
docs = knowledge_base.similarity_search(question)

In [12]:
system = "You are a Helpful assistant who answers questions based on the information user gives you"
context = docs[0].page_content + '\n\n' + docs[1].page_content + '\n\n' + docs[2].page_content + '\n\n' + docs[3].page_content + '\n\n' 
input = "Answer the question by looking at the context given and make sure that the answer is no more than 150 characters. both context and question are given below\n"

In [13]:
messages = [
    SystemMessage(content = system),
    HumanMessage(content = f"{input} CONTEXT : {context} \n QUESTION : {question}")
]

In [14]:
res = chat(messages)

In [15]:
messages.append(res)

In [16]:
res

AIMessage(content="This is Shivesh's resume.")

In [17]:
question = "Okay so how many internships did he do and name them?"

In [18]:
docs = knowledge_base.similarity_search(question)

In [19]:
context = docs[0].page_content + '\n\n' + docs[1].page_content + '\n\n' + docs[2].page_content + '\n\n' + docs[3].page_content + '\n\n' 
input = "Answer the question by looking at the context given and make sure that the answer is atleast 100 characters. both context and question are given below\n"

In [20]:
messages.append(HumanMessage(content = f"{input} CONTEXT : {context} \n QUESTION : {question}"))

In [21]:
res = chat(messages)

In [22]:
res

AIMessage(content='Shivesh did two internships. The first one was as a Remote Artificial Intelligence Intern at AiHello, and the second one was as a Data Science Intern at Locobuzz.')

In [23]:
question = "Okay tell me about his first internship?"

In [24]:
docs = knowledge_base.similarity_search(question)

In [25]:
context = docs[0].page_content + '\n\n' + docs[1].page_content + '\n\n' + docs[2].page_content + '\n\n' + docs[3].page_content + '\n\n' 
input = "Answer the question by looking at the context given and make sure that the answer is atleast 100 characters. both context and question are given below\n"

In [26]:
messages.append(HumanMessage(content = f"{input} CONTEXT : {context} \n QUESTION : {question}"))

In [27]:
res = chat(messages)

In [28]:
res

AIMessage(content="Shivesh's first internship was as a Data Science Intern at Locobuzz in Mumbai, Maharashtra, India from Mar 2021 to May 2021.")